In [133]:
import pandas as pd
import numpy as np

%pip install xlrd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\Albin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [134]:
gez = pd.read_csv("gezondheid-2020.csv", sep=";", index_col="Codering_3")
kwb = pd.read_excel("kwb-2019.xls", index_col="gwb_code_10")


In [135]:
# Find the amount rows that does not exists in gez and kwb
miss_aligned_codes = set(gez.index) - set(kwb.index)
print(
    f"{len(miss_aligned_codes)} codes are missing from on or the other dataset. That equates to approximately {len(miss_aligned_codes) / len(gez.index) * 100:.3f}% of the codes."
)

# Drop the rows where the code is missing
kwb = kwb.drop(miss_aligned_codes, errors="ignore")
gez = gez.drop(miss_aligned_codes, errors="igore")


524 codes are missing from on or the other dataset. That equates to approximately 3.022% of the codes.


## Explore kwb-dataset


In [136]:
kwb.sample(5)


,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,a_00_14,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
gwb_code_10,,,,,,,,,,,,,,,,,,,,,
BU19522001,19522001,Zuidbroek,Midden-Groningen,Buurt,BU19522001,1,3445,1770,1675,485,...,"1,1","1,0","2,0",572,550,22,9636,1,5,307
BU16800909,16800909,Verspreide huizen Oud-Annerveen,Aa en Hunze,Buurt,BU16800909,1,5,5,5,0,...,NaN,.,.,176,170,6,9655,1,5,24
WK016403,16403,Wijk 03 Hasseler Es,Hengelo,Wijk,WK016403,1,12350,6245,6105,2090,...,"0,6","0,6","9,1",363,363,0,.,.,3,1349
WK172308,172308,Wijk 08 Verspreide huizen,Alphen-Chaam,Wijk,WK172308,1,435,225,210,45,...,"1,4","2,1","1,7",1020,1011,9,.,.,5,70
BU03630107,3630107,Kalverdriehoek,Amsterdam,Buurt,BU03630107,1,265,150,110,10,...,"0,4","0,9","36,9",6,5,1,1012,1,1,9674


In [137]:
# Get all the columns with the proportion of missing values
kwb_missing_proportion = kwb.isna().sum().replace(0, np.nan).dropna() / len(kwb) * 100
print(kwb_missing_proportion)

# Drop the columns that have more than 50 percent missing values
columns_to_drop = kwb_missing_proportion[kwb_missing_proportion > 50].index
kwb = kwb.drop(columns=columns_to_drop)


a_inkont    100.000000
g_ink_po    100.000000
g_ink_pi    100.000000
p_ink_li    100.000000
p_ink_hi    100.000000
p_n_act     100.000000
p_hh_li     100.000000
p_hh_hi     100.000000
p_hh_lkk    100.000000
p_hh_osm    100.000000
g_wodief    100.000000
g_vernoo    100.000000
g_gewsek    100.000000
g_afs_hp      3.938296
g_afs_gs      3.938296
g_afs_kv      3.938296
ste_oad       0.011686
dtype: float64


In [138]:
# Make sure the columns are correctly dropped
kwb.isna().sum().replace(0, np.nan).dropna() / len(kwb) * 100


g_afs_hp    3.938296
g_afs_gs    3.938296
g_afs_kv    3.938296
ste_oad     0.011686
dtype: float64

## Explore gezondheid-dataset


In [139]:
gez.sample(5)


,ID,Leeftijd,Marges,WijkenEnBuurten,Perioden,Gemeentenaam_1,WekelijkseSporters_6,Eenzaam_28
Codering_3,,,,,,,,
BU02288051,13778,20300,MW00000,BU02288051,2020JJ00,Ede,36.7,41.0
BU00860902,34685,20300,MW00000,BU00860902,2020JJ00,Opsterland,48.9,37.1
BU05300800,21377,20300,MW00000,BU05300800,2020JJ00,Hellevoetsluis,47.7,43.2
BU03071903,2354,20300,MW00000,BU03071903,2020JJ00,Amersfoort,49.7,43.4
BU19305001,32222,20300,MW00000,BU19305001,2020JJ00,Nissewaard,47.7,43.1


In [140]:
# Drop unnecessary columns
gez = gez.drop(
    ["ID", "Leeftijd", "Marges", "Perioden", "Gemeentenaam_1", "WijkenEnBuurten"],
    axis=1,
    errors="ignore",
)


In [141]:
gez.info()


<class 'pandas.core.frame.DataFrame'>
Index: 16817 entries, GM1680 to BU01935210
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   WekelijkseSporters_6  16817 non-null  object
 1   Eenzaam_28            16817 non-null  object
dtypes: object(2)
memory usage: 394.1+ KB


In [142]:
# Fix wrong dtype on the columns
wrong_dtype = ["WekelijkseSporters_6", "Eenzaam_28"]
gez[wrong_dtype] = gez[wrong_dtype].replace(" .", np.nan)
gez[wrong_dtype] = gez[wrong_dtype].astype(float)


In [143]:
round(gez.describe(), 2)


,WekelijkseSporters_6,Eenzaam_28
count,16127.00,16127.00
mean,48.52,44.08
std,7.70,6.38
min,14.70,22.30
25%,43.30,39.40
50%,48.10,42.70
75%,53.50,47.70
max,75.40,79.50


In [144]:
gez.isna().sum() / len(gez) * 100


WekelijkseSporters_6    4.102991
Eenzaam_28              4.102991
dtype: float64

In [158]:
df = kwb.copy()
df[["sporting", "loneliness"]] = gez[["WekelijkseSporters_6", "Eenzaam_28"]]


In [160]:
df.to_csv("pre-processed-data.csv")
